<a href="https://colab.research.google.com/github/edsondamasceno/deep-learning-methods/blob/main/LeNet_5_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten, Input, Dropout
from keras.optimizers import Adam
from PIL import Image
import pandas as pd
import os
import cv2

In [ ]:
def load_train_data():
    X_train = np.load('/content/drive/My Drive/IEEE/NPY/X_train.npy')
    Y_train = np.load('/content/drive/My Drive/IEEE/NPY/Y_train.npy')
    return X_train, Y_train

In [ ]:
def load_test_data():
    X_test = np.load('/content/drive/My Drive/IEEE/NPY/X_test.npy')
    Y_test = np.load('/content/drive/My Drive/IEEE/NPY/Y_test.npy')
    return X_test, Y_test

In [ ]:
(X_train, y_train) = load_train_data()
(X_test, y_test) = load_test_data()

print(y_train[:1])
# pré-processamento de dados
X_train = X_train.reshape(-1, 1, 224, 224)/255.
X_test = X_test.reshape(-1, 1, 224, 224)/255.
Y_train = np_utils.to_categorical(y_train, num_classes=None)
Y_test = np_utils.to_categorical(y_test, num_classes=None)
print(Y_train[:1])

[[0.]]
[[1. 0.]]


In [ ]:
# Outra forma de construir sua CNN
_input = Input(shape=(1,224,224))

conv2d1 = Convolution2D(filters=32, kernel_size=5, padding='same', data_format='channels_first')(_input)
activation1 = Activation('relu')(conv2d1)
maxpooling1 = MaxPooling2D(pool_size=2, strides=2, padding='same', data_format='channels_first')(activation1)
dropout1 = Dropout(0.25)(maxpooling1)

conv2d2 = Convolution2D(filters=32, kernel_size=5, padding='same', data_format='channels_first')(dropout1)
activation2 = Activation('relu')(conv2d2)
maxpooling2 = MaxPooling2D(pool_size=2, strides=2, padding='same', data_format='channels_first')(activation2)

flatten = Flatten()(maxpooling2)

dense1 = Dense(1024)(flatten)
activation3 = Activation('relu')(dense1)

dense2 = Dense(2)(activation3)
activation4 = Activation('softmax')(dense2)

model_dense2_output = Model(inputs=_input, outputs=activation3)
model = Model(inputs=_input, outputs=activation4)
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 224, 224)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 224, 224)      832       
_________________________________________________________________
activation (Activation)      (None, 32, 224, 224)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 112, 112)      0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 112, 112)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 112, 112)      25632     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 112, 112)     

In [ ]:
adam = Adam(lr=0.01)
model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

print('Training ------------')
# Outra forma de treinar o modelo
model.fit(X_train, Y_train, epochs=200, batch_size=8)

print('\nTesting ------------')
# Avalie o modelo com as métricas que foram definidas anteriormente.
loss, accuracy = model.evaluate(X_test, Y_test)

print('\ntest loss: ', loss)
print('\ntest accuracy: ', accuracy)

path_save_train = '/content/drive/My Drive/IEEE/Data/train_LeNet-5.csv'
path_save_test = '/content/drive/My Drive/IEEE/Data/test_LeNet-5.csv'

X_train_xg = model_dense2_output.predict(X_train)
X_test_xg = model_dense2_output.predict(X_test)

print(np.array(X_train_xg).shape)
print(np.array(y_train).shape)
print(np.array(X_test_xg).shape)
print(np.array(Y_test).shape)

#Quantidade de dados de treinamento
z = np.concatenate([np.array(X_train_xg).reshape(566,1024),np.array(y_train).reshape(566,1)],axis=1)
z = pd.DataFrame(z)
z.to_csv(path_save_train,index=False)

#Quantidade de dados de teste
z = np.concatenate([np.array(X_test_xg).reshape(142,1024),np.array(y_test).reshape(142,1)],axis=1)
z = pd.DataFrame(z)
z.to_csv(path_save_test,index=False)

Training ------------
Epoch 1/200
71/71 [==============================] - 3s 36ms/step - loss: 17.8919 - accuracy: 0.5442
Epoch 2/200
71/71 [==============================] - 3s 35ms/step - loss: 0.6917 - accuracy: 0.5565
Epoch 3/200
71/71 [==============================] - 2s 35ms/step - loss: 0.6877 - accuracy: 0.5565
Epoch 4/200
71/71 [==============================] - 3s 36ms/step - loss: 0.6873 - accuracy: 0.5565
Epoch 5/200
71/71 [==============================] - 3s 35ms/step - loss: 0.6884 - accuracy: 0.5565
Epoch 6/200
71/71 [==============================] - 3s 35ms/step - loss: 0.6877 - accuracy: 0.5565
Epoch 7/200
71/71 [==============================] - 3s 36ms/step - loss: 0.6881 - accuracy: 0.5565
Epoch 8/200
71/71 [==============================] - 3s 36ms/step - loss: 0.6880 - accuracy: 0.5565
Epoch 9/200
71/71 [==============================] - 3s 35ms/step - loss: 0.6874 - accuracy: 0.5565
Epoch 10/200
71/71 [==============================] - 3s 35ms/step - loss: 0.